In [1]:
chrome_driver = './driver/chromedriver.exe'

In [2]:
URL = "https://play.google.com/store/apps/details?id=com.app.balaan.balaanboutique&hl=ko-KR"

In [3]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd

In [4]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

In [14]:
# 크롬 드라이버 세팅
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(chrome_driver, options = chrome_options)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
all_review_button_xpath = '//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[2]/div[2]/div/div[1]/div[1]/c-wiz[4]/section/div/div[2]/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element(By.XPATH,all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

C:\Users\hsty9\AppData\Local\Temp\ipykernel_2096\1432366863.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver, options = chrome_options)


스크롤 완료


In [15]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [16]:
driver.quit()

In [17]:
# html 데이터 저장
with open("../data/b_data_html.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

In [18]:
from tqdm import tqdm

In [19]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

100%|██████████████████████████████████████████████████████████████████████████████| 1766/1766 [01:28<00:00, 19.95it/s]


In [20]:
df = pd.DataFrame(dataset)
df.to_csv('../data/b_review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

In [21]:
df = pd.read_csv('../data/b_review_dataset.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20230517,2023,5,17,1,TEDDY,안경 판매자는 해외구매 제품인데 렌즈는 자기들 기술로 업그레이드했으니 반품시에는 2...
1,2,20230409,2023,4,9,3,DASTAN LEE (DASTAN),스마트폰 어플인데 스마트폰에서 상품 사진 및 설명등이 확대가 안됨. 진짜 불편함. ...
2,3,20210124,2021,1,24,3,강예지,다른 명품쇼핑 어플들에 비해 다양한 상품들이 정보량이 많은 것 같아 좋음. 타 어플...
3,4,20220204,2022,2,4,3,델비아,수많은 상품 스크롤내려가며 탐색중 맘에드는 상품눌러서 구경한후 뒤로가기 버튼누르면 ...
4,5,20200321,2020,3,21,1,Google 사용자,스타트업 응원차 기대하며 쇼핑했는데 너무 아쉽네요...결제한다고 30분이상 걸린건 ...
...,...,...,...,...,...,...,...,...
1761,1762,20220429,2022,4,29,1,제갈중섭,에휴....
1762,1763,20230530,2023,5,30,5,윤소원,Good~
1763,1764,20220519,2022,5,19,1,도니,비싸요
1764,1765,20210102,2021,1,2,1,sohyeun,버퍼링


In [22]:
df.sort_values(by='date', ascending=False, inplace=True)
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
1729,1730,20230914,2023,9,14,5,양혜승,자주 이용하지는 않지만 기분전환으로 조그만거 하나씩 사고 있어요..! 생일선물 살때...
1730,1731,20230914,2023,9,14,5,기쁨이,앱깔고 눈팅만 하다가 전부터 사고싶던게 특가로 떠서 샀어요ㅎㅎ산 물건에 비해 너무큰...
1731,1732,20230914,2023,9,14,5,빙글뱅글,매장가보다 저렴해서 놀랐어요!!!회원갑쿠폰쓰면 더저렴하고요!!구경하다보면 시간가는줄...
1732,1733,20230914,2023,9,14,5,Magda,상품보기 편하게 구분되어 있어서 구매하는 입장에서 쇼핑하기 조음 국내에서는 구하기 ...
1734,1735,20230914,2023,9,14,5,엄보미,인스타광고보고 접속했는데 사려고했던 아이템말고도 더 마음에 드는게 많아서 절제하느라...
...,...,...,...,...,...,...,...,...
1511,1512,20191023,2019,10,23,1,Google 사용자,앱 첫구매 만원할인이래서 받았는데 없음
1753,1754,20191020,2019,10,20,5,Google 사용자,좋네요
363,364,20191017,2019,10,17,5,Google 사용자,백화점에서 구할수없는 제품을 살수있어서 차별화된 제품을 구할수있다는것에 제일만족스럽...
478,479,20191015,2019,10,15,1,Google 사용자,하.. 진짜 발란 이러기 있나요 처음으로 발란을 통해서 구입을 하려고 했으나.. 첫...


In [23]:
df.reset_index(drop=True, inplace=True)
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1730,20230914,2023,9,14,5,양혜승,자주 이용하지는 않지만 기분전환으로 조그만거 하나씩 사고 있어요..! 생일선물 살때...
1,1731,20230914,2023,9,14,5,기쁨이,앱깔고 눈팅만 하다가 전부터 사고싶던게 특가로 떠서 샀어요ㅎㅎ산 물건에 비해 너무큰...
2,1732,20230914,2023,9,14,5,빙글뱅글,매장가보다 저렴해서 놀랐어요!!!회원갑쿠폰쓰면 더저렴하고요!!구경하다보면 시간가는줄...
3,1733,20230914,2023,9,14,5,Magda,상품보기 편하게 구분되어 있어서 구매하는 입장에서 쇼핑하기 조음 국내에서는 구하기 ...
4,1735,20230914,2023,9,14,5,엄보미,인스타광고보고 접속했는데 사려고했던 아이템말고도 더 마음에 드는게 많아서 절제하느라...
...,...,...,...,...,...,...,...,...
1761,1512,20191023,2019,10,23,1,Google 사용자,앱 첫구매 만원할인이래서 받았는데 없음
1762,1754,20191020,2019,10,20,5,Google 사용자,좋네요
1763,364,20191017,2019,10,17,5,Google 사용자,백화점에서 구할수없는 제품을 살수있어서 차별화된 제품을 구할수있다는것에 제일만족스럽...
1764,479,20191015,2019,10,15,1,Google 사용자,하.. 진짜 발란 이러기 있나요 처음으로 발란을 통해서 구입을 하려고 했으나.. 첫...


In [24]:
df.drop(columns='id', inplace=True)
df

,date,dateYear,dateMonth,dateDay,rating,userName,content
0,20230914,2023,9,14,5,양혜승,자주 이용하지는 않지만 기분전환으로 조그만거 하나씩 사고 있어요..! 생일선물 살때...
1,20230914,2023,9,14,5,기쁨이,앱깔고 눈팅만 하다가 전부터 사고싶던게 특가로 떠서 샀어요ㅎㅎ산 물건에 비해 너무큰...
2,20230914,2023,9,14,5,빙글뱅글,매장가보다 저렴해서 놀랐어요!!!회원갑쿠폰쓰면 더저렴하고요!!구경하다보면 시간가는줄...
3,20230914,2023,9,14,5,Magda,상품보기 편하게 구분되어 있어서 구매하는 입장에서 쇼핑하기 조음 국내에서는 구하기 ...
4,20230914,2023,9,14,5,엄보미,인스타광고보고 접속했는데 사려고했던 아이템말고도 더 마음에 드는게 많아서 절제하느라...
...,...,...,...,...,...,...,...
1761,20191023,2019,10,23,1,Google 사용자,앱 첫구매 만원할인이래서 받았는데 없음
1762,20191020,2019,10,20,5,Google 사용자,좋네요
1763,20191017,2019,10,17,5,Google 사용자,백화점에서 구할수없는 제품을 살수있어서 차별화된 제품을 구할수있다는것에 제일만족스럽...
1764,20191015,2019,10,15,1,Google 사용자,하.. 진짜 발란 이러기 있나요 처음으로 발란을 통해서 구입을 하려고 했으나.. 첫...


In [25]:
df.to_csv('../data/balaan_google.csv', encoding = 'utf-8-sig')